In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import time
import pdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# Clustering
from sklearn.datasets import make_biclusters, make_checkerboard, make_classification
from sklearn.datasets import samples_generator as sg
from sklearn.cluster.bicluster import SpectralCoclustering, SpectralBiclustering
from sklearn.metrics import consensus_score

# Corr
from scipy.stats import spearmanr, pearsonr
from numpy import corrcoef

# Utils
file_path = os.getcwd()  # os.path.dirname(os.path.relpath(__file__))
utils_path = os.path.abspath(os.path.join(file_path, '..', '..', 'utils_py'))
sys.path.append(utils_path)
import utils_all as utils

# Feature importance
pfi_path = os.path.abspath(os.path.join(file_path, '..', 'pfi'))
sys.path.append(pfi_path)
import pfi
import pfi_utils

import warnings
warnings.filterwarnings('ignore')

SEED=0

Using TensorFlow backend.


In [2]:
path = 'results_P1B1_cor0.85'
APP = 'P1B1'

In [3]:
rfs_fi = pd.read_csv(os.path.join(file_path, path, f'{APP}_rf_fi_score.csv'))
rfv_fi = pd.read_csv(os.path.join(file_path, path, f'{APP}_rf_fi_var.csv'))
rf_fi  = pd.read_csv(os.path.join(file_path, path, f'{APP}_rf_fi.csv'))

# fip = pd.read_csv(os.path.join(file_path, path, f'{APP}_rf_fi_score_p.csv'))
fipmap = pd.read_csv(os.path.join(file_path, path, f'{APP}_rf_fi_score_pmap.csv'))